# Training and Evaluation: Individual Models

================================================================================
PURPOSE: Train and evaluate classifiers on features from each model separately
================================================================================

This notebook trains multiple classifiers on Context Tree features extracted
from individual transformer models. Each model (BERT, RoBERTa, DeBERTa, XLNet)
is evaluated separately to assess their individual performance on the clarity
and evasion classification tasks.

**Workflow:**
1. Load features from Google Drive (saved by 02_feature_extraction_separate.ipynb)
2. Train multiple classifiers on each model's features
3. Evaluate on Dev set (model selection and hyperparameter tuning)
4. Save predictions and probabilities for further analysis
5. Generate comprehensive results tables and evaluation plots

**Classifiers:**
- Logistic Regression
- Linear Support Vector Classifier (LinearSVC)
- Random Forest
- XGBoost
- LightGBM

**Output:** 
- Predictions (hard labels) and probabilities saved to Google Drive
- Results tables comparing classifiers for each model/task combination
- Evaluation plots (confusion matrices, PR curves, ROC curves)
- Results metadata saved for final summary generation


In [ ]:
# ============================================================================
# SETUP: Repository Clone, Drive Mount, and Path Configuration
# ============================================================================
# This cell performs minimal setup required for the notebook to run:
# 1. Clones repository from GitHub (if not already present)
# 2. Mounts Google Drive for persistent data storage
# 3. Configures Python paths and initializes StorageManager
# 4. Loads data splits and features created in previous notebooks

import shutil
import os
import subprocess
import time
import requests
import zipfile
import sys
from pathlib import Path
from google.colab import drive
import numpy as np

# Repository configuration
repo_dir = '/content/semeval-context-tree-modular'
repo_url = 'https://github.com/EonTechie/semeval-context-tree-modular.git'
zip_url = 'https://github.com/EonTechie/semeval-context-tree-modular/archive/refs/heads/main.zip'

# Clone repository (if not already present)
if not os.path.exists(repo_dir):
    print("Cloning repository from GitHub...")
    max_retries = 2
    clone_success = False
    
    for attempt in range(max_retries):
        try:
            result = subprocess.run(
                ['git', 'clone', repo_url],
                cwd='/content',
                capture_output=True,
                text=True,
                timeout=60
            )
            if result.returncode == 0:
                print("Repository cloned successfully via git")
                clone_success = True
                break
            else:
                if attempt < max_retries - 1:
                    time.sleep(3)
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(3)
    
    # Fallback: Download as ZIP if git clone fails
    if not clone_success:
        print("Git clone failed. Downloading repository as ZIP archive...")
        zip_path = '/tmp/repo.zip'
        try:
            response = requests.get(zip_url, stream=True, timeout=60)
            response.raise_for_status()
            with open(zip_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content')
            extracted_dir = '/content/semeval-context-tree-modular-main'
            if os.path.exists(extracted_dir):
                os.rename(extracted_dir, repo_dir)
            os.remove(zip_path)
            print("Repository downloaded and extracted successfully")
        except Exception as e:
            raise RuntimeError(f"Failed to obtain repository: {e}")

# Mount Google Drive (if not already mounted)
try:
    drive.mount('/content/drive', force_remount=False)
except Exception:
    pass  # Already mounted

# Configure paths
BASE_PATH = Path('/content/semeval-context-tree-modular')
DATA_PATH = Path('/content/drive/MyDrive/semeval_data')
sys.path.insert(0, str(BASE_PATH))

# Initialize StorageManager and load required modules
from src.storage.manager import StorageManager
from src.models.trainer import train_and_evaluate
from src.models.classifiers import get_classifier_dict

storage = StorageManager(
    base_path=str(BASE_PATH),
    data_path=str(DATA_PATH),
    github_path=str(BASE_PATH)
)

# Load data splits for label extraction
train_ds = storage.load_split('train')
dev_ds = storage.load_split('dev')

print("Setup complete")
print(f"  Repository: {BASE_PATH}")
print(f"  Data storage: {DATA_PATH}")
print(f"  Train samples: {len(train_ds)}")
print(f"  Dev samples: {len(dev_ds)}")


In [ ]:
# ============================================================================
# CONFIGURE MODELS, TASKS, AND CLASSIFIERS
# ============================================================================
# Defines the models to evaluate, tasks to perform, and classifiers to train
# Label mappings are defined for clarity (3-class) and evasion (9-class) tasks

MODELS = ['bert', 'roberta', 'deberta', 'xlnet']
TASKS = ['clarity', 'evasion']

# Label mappings for each task
CLARITY_LABELS = ['Clear Reply', 'Ambiguous', 'Clear Non-Reply']
EVASION_LABELS = ['Direct Answer', 'Partial Answer', 'Implicit Answer', 
                  'Uncertainty', 'Refusal', 'Clarification', 
                  'Question', 'Topic Shift', 'Other']

# Initialize classifiers with fixed random seed for reproducibility
classifiers = get_classifier_dict(random_state=42)

print("Configuration:")
print(f"  Models: {MODELS}")
print(f"  Tasks: {TASKS}")
print(f"  Classifiers: {list(classifiers.keys())}")
print(f"  Clarity labels: {len(CLARITY_LABELS)} classes")
print(f"  Evasion labels: {len(EVASION_LABELS)} classes")


In [ ]:
# ============================================================================
# TRAIN AND EVALUATE CLASSIFIERS FOR EACH MODEL AND TASK
# ============================================================================
# Iterates through each model and task, trains all classifiers, and evaluates
# on the Dev set. Results are saved for later analysis and final summary generation.

all_results = {}

for model in MODELS:
    print(f"\n{'='*80}")
    print(f"MODEL: {model.upper()}")
    print(f"{'='*80}")
    
    all_results[model] = {}
    
    for task in TASKS:
        print(f"\n{'='*60}")
        print(f"TASK: {task.upper()}")
        print(f"{'='*60}")
        
        # Select appropriate label list and dataset key for this task
        if task == 'clarity':
            label_list = CLARITY_LABELS
            label_key = 'clarity_label'
        else:  # evasion
            label_list = EVASION_LABELS
            label_key = 'evasion_label'
        
        # Load features from persistent storage
        print("Loading features from Google Drive...")
        X_train = storage.load_features(model, task, 'train')
        X_dev = storage.load_features(model, task, 'dev')
        
        # Extract labels from dataset splits
        y_train = np.array([train_ds[i][label_key] for i in range(len(train_ds))])
        y_dev = np.array([dev_ds[i][label_key] for i in range(len(dev_ds))])
        
        print(f"  Train: {X_train.shape[0]} samples, {X_train.shape[1]} features")
        print(f"  Dev: {X_dev.shape[0]} samples, {X_dev.shape[1]} features")
        
        # Train all classifiers and evaluate on Dev set
        # This function handles training, prediction, metric computation, and visualization
        results = train_and_evaluate(
            X_train, y_train, X_dev, y_dev,
            label_list=label_list,
            task_name=f"{model}_{task}",
            classifiers=classifiers,
            random_state=42,
            print_report=True,      # Print detailed classification report
            print_table=True,       # Print results comparison table
            create_plots=True,      # Generate confusion matrices and PR/ROC curves
            save_plots_dir=str(DATA_PATH / 'plots')
        )
        
        # Save predictions and probabilities to persistent storage
        # These will be used for further analysis and final summary generation
        for classifier_name, result in results.items():
            # Save hard label predictions
            storage.save_predictions(
                result['dev_pred'],
                model, classifier_name, task, 'dev'
            )
            
            # Save probability distributions (if classifier supports it)
            if result['dev_proba'] is not None:
                storage.save_probabilities(
                    result['dev_proba'],
                    model, classifier_name, task, 'dev'
                )
        
        all_results[model][task] = results
        
        # Save results summary to metadata for final summary generation
        experiment_id = f"{model}_{task}_separate"
        storage.save_results({
            'model': model,
            'task': task,
            'results': {
                name: {
                    'metrics': res['metrics'],
                    'n_train': len(y_train),
                    'n_dev': len(y_dev)
                }
                for name, res in results.items()
            }
        }, experiment_id)

print(f"\n{'='*80}")
print("Training and evaluation complete for all models and tasks")
print(f"{'='*80}")
print("\nSummary:")
print("  - All classifiers trained and evaluated on Dev set")
print("  - Predictions and probabilities saved to Google Drive")
print("  - Results tables and plots generated")
print("  - Metadata saved for final summary generation")
